In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../")

In [3]:
import torch.nn as nn

from data.input_transforms import InputTransform
from data.cifar import CIFAR
from data.dataloader import get_dataloader
from models.optimizer import optimizer, scheduler
from networks.twolayernet import TwolayerNet
from networks.convnet import ConvNet
from utils.viz import plot_multiple_quantities_by_time
from main import train, evaluate

In [4]:
def experiment(train_transforms, valid_transforms, network, num_epochs, batch_size, opt_args, sch_args=None):
    
    print(train_transforms)
    print(num_epochs)
    print(batch_size)
    print(opt_args)
    print(sch_args)
    
    # define the datasets
    train_transform = InputTransform(train_transforms)
    valid_transform = InputTransform(valid_transforms)
    train_dataset = CIFAR(root="../datasets/CIFAR-10/", mode="train", transform=train_transform)
    valid_dataset = CIFAR(root="../datasets/CIFAR-10/", mode="valid", transform=valid_transform)

    # obtain the train dataloader
    train_loader = get_dataloader(train_dataset, train=True, batch_size=batch_size, num_workers=0)
    valid_loader = get_dataloader(valid_dataset, train=False, batch_size=batch_size, num_workers=0)
    
    if network == "tln":
        net = TwolayerNet(num_inputs=3 * 32 * 32, num_hidden=512, num_classes=10)
    elif network == "cln":
        net = ConvNet(in_channels=3, num_classes=10)
    else:
        raise ValueError()
    arch = type(net).__name__

    # define the optimizer and scheduler
    opt = optimizer(model_params=net.parameters(), **opt_args)
    sch = None
    if sch_args is not None:
        sch = scheduler(opt=opt, **sch_args)

    # define the loss function
    loss_fn = nn.CrossEntropyLoss()

    # train the model
    epochs, train_losses, train_metrics, valid_losses, valid_metrics = train(
        net, loss_fn, train_loader, valid_loader, num_epochs=num_epochs, opt=opt, sch=sch,
    )

    # plot training curves
    plot_multiple_quantities_by_time(
        quantities=[train_losses["loss"], valid_losses["loss"]],
        time=epochs,
        labels=["Train", "Validation"],
        title=f"{arch} Loss curves",
        show=True,
        save=False,
        save_path="./results/loss_plot.png",
        ylabel="Loss",
    )
    plot_multiple_quantities_by_time(
        quantities=[train_metrics["accuracy"], valid_metrics["accuracy"]],
        time=epochs,
        labels=["Train", "Validation"],
        title=f"{arch} Accuracy curves",
        show=True,
        save=False,
        save_path="./results/accu_plot.png",
        ylabel="Accuracy",
    )

In [18]:
train_transforms = [
    {
        "name": "ToTensor",
        "args": {},
    },
    {
        "name": "Normalize",
        "args": {"mean": (0.5, 0.5, 0.5), "std": (0.5, 0.5, 0.5)},
    },
    {
        "name": "RandomAffine",
        "args": {"degrees": 10, "translate": (0.1, 0.1), "scale": (0.8, 1.2)},
    },
    {
        "name": "RandomHorizontalFlip",
        "args": {"p": 0.5},
    },
    {
        "name": "GaussianBlur",
        "args": {"kernel_size": 3},
    },
]
valid_transforms = [
    {
        "name": "ToTensor",
        "args": {},
    },
    {
        "name": "Normalize",
        "args": {"mean": (0.5, 0.5, 0.5), "std": (0.5, 0.5, 0.5)},
    },
]

network = "tln"
num_epochs = 200
batch_size = 128
# opt_args = {"name": "SGD", "lr": 3e-4, "momentum": 0.9}
opt_args = {"name": "Adam", "lr": 1e-4}
sch_args = {"name": "MultiStepLR", "milestones": [50, 100, 150], "gamma": 0.5}

experiment(train_transforms, valid_transforms, network, num_epochs, batch_size, opt_args, sch_args=sch_args)

[{'name': 'ToTensor', 'args': {}}, {'name': 'Normalize', 'args': {'mean': (0.5, 0.5, 0.5), 'std': (0.5, 0.5, 0.5)}}, {'name': 'RandomAffine', 'args': {'degrees': 10, 'translate': (0.1, 0.1), 'scale': (0.8, 1.2)}}, {'name': 'RandomHorizontalFlip', 'args': {'p': 0.5}}, {'name': 'GaussianBlur', 'args': {'kernel_size': 3}}]
200
128
{'name': 'Adam', 'lr': 0.0001}
{'name': 'MultiStepLR', 'milestones': [50, 100, 150], 'gamma': 0.5}
Selecting 40000 for train.
Selecting 10000 for valid.


Evaluate: Epoch [1/200]: 100%|██████████████████████████████| 79/79 [00:00<00:00, 131.15it/s]                                              


VALID 	: Summary: Loss: 1.9418 Accuracy: 0.3053
TRAIN 	: Summary: Loss: 2.0828 Accuracy: 0.2317


Evaluate: Epoch [2/200]: 100%|██████████████████████████████| 79/79 [00:00<00:00, 127.64it/s]                                              


VALID 	: Summary: Loss: 1.8783 Accuracy: 0.3363
TRAIN 	: Summary: Loss: 1.9528 Accuracy: 0.3003


...

VALID 	: Summary: Loss: 1.2179 Accuracy: 0.5672
TRAIN 	: Summary: Loss: 1.2990 Accuracy: 0.5355


Evaluate: Epoch [152/200]: 100%|██████████████████████████████| 79/79 [00:00<00:00, 128.18it/s]                                            


VALID 	: Summary: Loss: 1.2193 Accuracy: 0.5651
TRAIN 	: Summary: Loss: 1.2973 Accuracy: 0.5361


Training: Epoch [153/200]:  34%|██████████                    | 105/313 [00:04<00:08, 25.76it/s]                                           


KeyboardInterrupt: 